In [1]:
from loguru import logger as log

import sys

sys.path.append('../')

In [2]:
import numpy as np
from ortools.sat.python import cp_model

from vrp_study.data_model import Tariff, Cargo, Node
from vrp_study.data_model import TariffCost
from vrp_study.routing_manager import PDRoutingManager
from vrp_study.ortools_routing_model.routing_model import find_optimal_paths

In [3]:
benchmark_type = 'pdptw1000'
name = 'LRC1_10_7.txt'

In [4]:
from typing import Optional

tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [5]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


In [6]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1821, service_time=0, coordinates=(250, 250))

In [7]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [8]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [ ]:
from vrp_study.configs import ModelConfig


routing_manager = PDRoutingManager(
    distance_matrix = distance_matrix,
    time_matrix = time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

# routing_manager.distance_matrix = distance_matrix
# routing_manager.time_matrix = time_matrix


routing_manager = routing_manager.build()

In [10]:
from vrp_study.init_solution_builder.solution_builder import SolutionBuilder

# sol = find_optimal_paths(routing_manager, SolutionBuilder())
sol = find_optimal_paths(routing_manager)

2025-07-02 16:51:32.754 | INFO     | vrp_study.ortools_routing_model.routing_model:find_optimal_paths:435 - problem size: 1047
2025-07-02 16:51:32.755 | INFO     | vrp_study.ortools_routing_model.routing_model:do_solve:373 - Начало создания модели
2025-07-02 16:51:32.756 | INFO     | vrp_study.ortools_routing_model.routing_model:add_distance_dimension:293 - Добавление размерности для расстояния
2025-07-02 16:51:32.757 | INFO     | vrp_study.ortools_routing_model.routing_model:add_count_dimension:329 - Добавление размерности для расстояния
2025-07-02 16:51:32.758 | INFO     | vrp_study.ortools_routing_model.routing_model:add_pick_up_and_delivery:263 - Добавление ограничения для порядка доставки
2025-07-02 16:51:32.761 | INFO     | vrp_study.ortools_routing_model.routing_model:add_vehicles_cost:237 - Добавление стоимостей машин
2025-07-02 16:51:32.762 | INFO     | vrp_study.ortools_routing_model.routing_model:add_time_window:188 - add time
2025-07-02 16:51:32.764 | INFO     | vrp_study.o

In [11]:
len(cargos)

523

In [12]:
80 * 79

6320

In [13]:
#  best_score: 107
# 2025-06-02 14:45:05,380 - routing_model [INFO] - best_score: 46228.27

In [14]:
# est_score: 8
# 2025-06-02 14:25:30,179 - routing_model [INFO] - best_score: 1979.21

In [15]:
total_l = 0
for s in sol[0]:
    if len(s) > 0:
        # print(s)
        # a = routing_manager.nodes()[s[i]]
        # b = routing_manager.nodes()[s[i+1]]
        
        l = sum(routing_manager.get_distance(routing_manager.nodes()[s[i]], routing_manager.nodes()[s[i + 1]]) for i in range(len(s) - 1))
        # print(s, l)
        total_l += l
total_l

58418.89676549103

In [ ]:
routing_manager.

In [16]:
for crg in cargos:
    in1 = routing_manager._node_to_inner_node[crg.nodes[0]]
    in2 = routing_manager._node_to_inner_node[crg.nodes[1]]

    start = None
    end = None
    in_time = None
    out_time = None

    for i, s in enumerate(sol[0]):
        s_time = sol[1][i]
        if len(s) == 0:
            continue

        for order, i in enumerate(s):
            if i == in1.id:
                start = i
                in_time = s_time[order]
            if i == in2.id:
                end = i
                out_time = s_time[order]

    assert start is not None and end is not None and start < end, (start, end)

    if not (crg.nodes[0].start_time <= in_time[0] and crg.nodes[0].end_time >= in_time[1]):
        print(in_time, crg.nodes[0].start_time, crg.nodes[0].end_time)
    if not (crg.nodes[1].start_time <= out_time[0] and crg.nodes[1].end_time >= out_time[1]):
        print(out_time, crg.nodes[0].start_time, crg.nodes[0].end_time)

AttributeError: 'RoutingManager' object has no attribute '_node_to_inner_node'